In [1]:
!pip install opendatasets
import opendatasets as od
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False)
base_model.trainable = False
od.download("https://www.kaggle.com/emmarex/plantdisease")

inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)

outputs = keras.layers.Dense(units = 15, activation='softmax')(x)
model = keras.Model(inputs, outputs)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

     |████████████████████████████████| 97 kB 1.9 MB/s             
     |████████████████████████████████| 58 kB 1.2 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 78 kB 13.4 MB/s            
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73053 sha256=9a8798a5a2fdb68cfac57b3ad9d503927c1bc4d772f6635fcfd64a0e3c54e6dc
  Stored in directory: /root/.cache/pip/wheels/77/47/e4/44a4ba1b7dfd53faaa35f59f1175e123b213ff401a8a56876b
Successfully built kaggle
58892288/58889256 [==============================] - 0s 0us/step
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  hananmiftah


Your Kaggle Key:

  ································


  2%|▏         | 14.0M/658M [00:00<00:04, 143MB/s]

100%|██████████| 658M/658M [00:04<00:00, 153MB/s] 


In [4]:
!pip install split-folders
import splitfolders
splitfolders.ratio('plantdisease/plantvillage/PlantVillage/', output="output", seed=1337, ratio=(.8, 0.1,0.1))

Copying files: 20639 files [00:02, 9892.42 files/s] 


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# create a data generator
datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True) # we don't expect Bo to be upside-down so we will not flip vertically

In [7]:
train_it = datagen.flow_from_directory('output/train/', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='categorical', 
                                       batch_size=8)
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('output/val/', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode='categorical', 
                                      batch_size=8)

Found 16504 images belonging to 15 classes.
Found 2058 images belonging to 15 classes.


In [ ]:
model.fit(train_it, steps_per_epoch=train_it.samples/train_it.batch_size, validation_data=valid_it, validation_steps=valid_it.samples/valid_it.batch_size, epochs=20)

Epoch 1/20
2063/2063 [==============================] - 203s 99ms/step - loss: 0.9354 - accuracy: 0.7298 - val_loss: 0.5034 - val_accuracy: 0.8440
Epoch 2/20
2063/2063 [==============================] - 200s 97ms/step - loss: 0.4473 - accuracy: 0.8612 - val_loss: 0.3927 - val_accuracy: 0.8819
Epoch 3/20
2063/2063 [==============================] - 200s 97ms/step - loss: 0.4001 - accuracy: 0.8772 - val_loss: 0.3855 - val_accuracy: 0.8853
Epoch 4/20
 370/2063 [====>.........................] - ETA: 2:25 - loss: 0.3842 - accuracy: 0.8807